# Ch21. Natural Language Processing

### n-Gram Language Models

In [1]:
def fix_unicode(text: str) -> str:
    return text.replace(u"\u2019", "'")

In [2]:
import re
from bs4 import BeautifulSoup
import requests

url = "https://www.oreilly.com/ideas/what-is-data-science"
html = requests.get(url).text
soup = BeautifulSoup(html, 'html5lib')

content = soup.find("div", "main-post-radar-content")
regex = r"[\w']+|[\.]"                       # matches a word or a period

document = []

for paragraph in content("p"):
    words = re.findall(regex, fix_unicode(paragraph.text))
    document.extend(words)

In [3]:
document[:10]

['Weâ',
 've',
 'all',
 'heard',
 'it',
 'according',
 'to',
 'Hal',
 'Varian',
 'statistics']

In [4]:
len(document)

5225

In [5]:
document[1:5]

['ve', 'all', 'heard', 'it']

In [6]:
from collections import defaultdict

transitions = defaultdict(list)
for prev, current in zip(document, document[1:]):
    transitions[prev].append(current)

In [7]:
import itertools

dict(itertools.islice(transitions.items(), 10))

{'Weâ': ['ve', 're', 've', 're', 've'],
 've': ['all',
  'ever',
  'taken',
  'made',
  'seen',
  'ever',
  'parsed',
  'all',
  'heard',
  'analyzed',
  'seen',
  'collected',
  'all',
  'gotten',
  'just'],
 'all': ['heard',
  'the',
  'of',
  'of',
  'carefully',
  'the',
  'in',
  'equipment',
  'youâ',
  'heard',
  'data',
  'of',
  'heard',
  'locked',
  'at',
  'trying',
  'aspects',
  'tapped',
  'The'],
 'heard': ['it', 'a', 'â', 'the'],
 'it': ['according',
  'to',
  'does',
  'to',
  'to',
  '.',
  'into',
  'tell',
  'comes',
  'and',
  'goes',
  'possible',
  'and',
  '.',
  '.',
  'well',
  'simpler',
  'necessary',
  'and',
  'onto',
  'much',
  '.',
  'might',
  'easy',
  'easier',
  'arrives',
  'takes',
  'complements',
  'comes',
  '.',
  '.',
  'up',
  '.',
  'look',
  'involves',
  'isnâ',
  'tell',
  'started',
  'was',
  '.',
  'â',
  'all',
  'to',
  'to',
  'to',
  'to',
  'â',
  'appears'],
 'according': ['to'],
 'to': ['Hal',
  'a',
  'a',
  'rip',
  'CDDB',


In [8]:
import random

def generate_using_bigrams() -> str:
    current = "."   # this means the next word will start a sentence
    result = []
    while True:
        next_word_candidates = transitions[current]    # bigrams (current, _)
        current = random.choice(next_word_candidates)  # choose one at random
        result.append(current)                         # append it to results
        if current == ".": return " ".join(result)     # if "." we're done

In [9]:
generate_using_bigrams()

'According to data productsâ .'

In [10]:
trigram_transitions = defaultdict(list)
starts = []

for prev, current, next in zip(document, document[1:], document[2:]):

    if prev == ".":              # if the previous "word" was a period
        starts.append(current)   # then this is a start word

    trigram_transitions[(prev, current)].append(next)

In [11]:
def generate_using_trigrams() -> str:
    current = random.choice(starts)   # choose a random starting word
    prev = "."                        # and precede it with a '.'
    result = [current]
    while True:
        next_word_candidates = trigram_transitions[(prev, current)]
        next_word = random.choice(next_word_candidates)

        prev, current = current, next_word
        result.append(current)

        if current == ".":
            return " ".join(result)

In [12]:
generate_using_trigrams()

'â Whether humans or software decided to ignore anomalous data it appears that data might mean itâ s easy to notice that one advertisement for R in a sea of data wherever they go .'

### Grammars

In [13]:
from typing import List, Dict

# Type alias to refer to grammars later
Grammar = Dict[str, List[str]]

grammar = {
    "_S"  : ["_NP _VP"], # NP: noun phrase, VP: verb phrase
    "_NP" : ["_N",
             "_A _NP _P _A _N"],
    "_VP" : ["_V",
             "_V _NP"],
    "_N"  : ["data science", "Python", "regression"],
    "_A"  : ["big", "linear", "logistic"],
    "_P"  : ["about", "near"],
    "_V"  : ["learns", "trains", "tests", "is"]
}

In [14]:
def is_terminal(token: str) -> bool:
    return token[0] != "_"

In [15]:
def expand(grammar: Grammar, tokens: List[str]) -> List[str]:
    for i, token in enumerate(tokens):
        # If this is a terminal token, skip it.
        if is_terminal(token): continue

        # Otherwise, it's a nonterminal token,
        # so we need to choose a replacement at random.
        replacement = random.choice(grammar[token])

        if is_terminal(replacement):
            tokens[i] = replacement
        else:
            # Replacement could be, e.g., "_NP _VP", so we need to
            # split it on spaces and splice it in.
            tokens = tokens[:i] + replacement.split() + tokens[(i+1):]

        # Now call expand on the new list of tokens.
        return expand(grammar, tokens)

    # If we get here, we had all terminals and are done.
    return tokens

In [16]:
def generate_sentence(grammar: Grammar) -> List[str]:
    return expand(grammar, ["_S"])

In [17]:
generate_sentence(grammar)

['data science', 'is']

### An Aside: Gibbs Sampling

In [18]:
from typing import Tuple
import random

def roll_a_die() -> int:
    return random.choice([1, 2, 3, 4, 5, 6])

def direct_sample() -> Tuple[int, int]:
    d1 = roll_a_die()
    d2 = roll_a_die()
    return d1, d1 + d2

In [19]:
def random_y_given_x(x: int) -> int:
    """equally likely to be x + 1, x + 2, ... , x + 6"""
    return x + roll_a_die()

In [20]:
def random_x_given_y(y: int) -> int:
    if y <= 7:
        # if the total is 7 or less, the first die is equally likely to be
        # 1, 2, ..., (total - 1)
        return random.randrange(1, y)
    else:
        # if the total is 7 or more, the first die is equally likely to be
        # (total - 6), (total - 5), ..., 6
        return random.randrange(y - 6, 7)

In [21]:
def gibbs_sample(num_iters: int = 100) -> Tuple[int, int]:
    x, y = 1, 2 # doesn't really matter
    for _ in range(num_iters):
        x = random_x_given_y(y)
        y = random_y_given_x(x)
    return x, y

In [22]:
def compare_distributions(num_samples: int = 1000) -> Dict[int, List[int]]:
    counts = defaultdict(lambda: [0, 0])
    for _ in range(num_samples):
        counts[gibbs_sample()][0] += 1
        counts[direct_sample()][1] += 1
    return counts

### Topic Modeling

In [23]:
# if you give it weights [1, 1, 3], then one-fifth of the time it will return 0, 
# one-fifth of the time it will return 1, 
# and three-fifths of the time it will return 2.
def sample_from(weights: List[float]) -> int:
    """returns i with probability weights[i] / sum(weights)"""
    total = sum(weights)
    rnd = total * random.random()      # uniform between 0 and total
    for i, w in enumerate(weights):
        rnd -= w                       # return the smallest i such that
        if rnd <= 0: return i          # weights[0] + ... + weights[i] >= rnd

In [24]:
from collections import Counter

# Draw 1000 times and count
draws = Counter(sample_from([0.1, 0.1, 0.8]) for _ in range(1000))
assert 10 < draws[0] < 190   # should be ~10%, this is a really loose test
assert 10 < draws[1] < 190   # should be ~10%, this is a really loose test
assert 650 < draws[2] < 950  # should be ~80%, this is a really loose test
assert draws[0] + draws[1] + draws[2] == 1000

In [25]:
documents = [
    ["Hadoop", "Big Data", "HBase", "Java", "Spark", "Storm", "Cassandra"],
    ["NoSQL", "MongoDB", "Cassandra", "HBase", "Postgres"],
    ["Python", "scikit-learn", "scipy", "numpy", "statsmodels", "pandas"],
    ["R", "Python", "statistics", "regression", "probability"],
    ["machine learning", "regression", "decision trees", "libsvm"],
    ["Python", "R", "Java", "C++", "Haskell", "programming languages"],
    ["statistics", "probability", "mathematics", "theory"],
    ["machine learning", "scikit-learn", "Mahout", "neural networks"],
    ["neural networks", "deep learning", "Big Data", "artificial intelligence"],
    ["Hadoop", "Java", "MapReduce", "Big Data"],
    ["statistics", "R", "statsmodels"],
    ["C++", "deep learning", "artificial intelligence", "probability"],
    ["pandas", "R", "Python"],
    ["databases", "HBase", "Postgres", "MySQL", "MongoDB"],
    ["libsvm", "regression", "support vector machines"]
]

In [26]:
K = 4

In [27]:
# How many times each topic is assigned to each document:
# a list of Counters, one for each document
document_topic_counts = [Counter() for _ in documents]

In [28]:
document_topic_counts

[Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter(),
 Counter()]

In [29]:
# How many times each word is assigned to each topic:
# a list of Counters, one for each topic
topic_word_counts = [Counter() for _ in range(K)]

In [30]:
# The total number of words assigned to each topic:
# a list of numbers, one for each topic
topic_counts = [0 for _ in range(K)]

In [31]:
# The total number of words contained in each document:
# a list of numbers, one for each document
document_lengths = [len(document) for document in documents]

In [32]:
# The number of distinct words:
distinct_words = set(word for document in documents for word in document)
W = len(distinct_words)

In [33]:
# And the number of documents:
D = len(documents)

In [34]:
#  the number of words in documents[3] associated with topic 1 as follows:
document_topic_counts[3][1]

0

In [35]:
# And we can find the number of times nlp is associated with topic 2 as follows:
topic_word_counts[2]["nlp"]

0

In [36]:
def p_topic_given_document(topic: int, d: int, alpha: float = 0.1) -> float:
    """
    The fraction of words in document 'd'
    that are assigned to 'topic' (plus some smoothing)
    """
    return ((document_topic_counts[d][topic] + alpha) /
            (document_lengths[d] + K * alpha))

def p_word_given_topic(word: str, topic: int, beta: float = 0.1) -> float:
    """
    The fraction of words assigned to 'topic'
    that equal 'word' (plus some smoothing)
    """
    return ((topic_word_counts[topic][word] + beta) /
            (topic_counts[topic] + W * beta))

In [37]:
def topic_weight(d: int, word: str, k: int) -> float:
    """
    Given a document and a word in that document,
    return the weight for the kth topic
    """
    return p_word_given_topic(word, k) * p_topic_given_document(k, d)

def choose_new_topic(d: int, word: str) -> int:
    return sample_from([topic_weight(d, word, k)
                        for k in range(K)])

In [38]:
random.seed(0)
document_topics = [[random.randrange(K) for word in document]
                   for document in documents]

for d in range(D):
    for word, topic in zip(documents[d], document_topics[d]):
        document_topic_counts[d][topic] += 1
        topic_word_counts[topic][word] += 1
        topic_counts[topic] += 1

In [39]:
import tqdm

for iter in tqdm.trange(1000):
    for d in range(D):
        for i, (word, topic) in enumerate(zip(documents[d],
                                              document_topics[d])):

            # remove this word / topic from the counts
            # so that it doesn't influence the weights
            document_topic_counts[d][topic] -= 1
            topic_word_counts[topic][word] -= 1
            topic_counts[topic] -= 1
            document_lengths[d] -= 1

            # choose a new topic based on the weights
            new_topic = choose_new_topic(d, word)
            document_topics[d][i] = new_topic

            # and now add it back to the counts
            document_topic_counts[d][new_topic] += 1
            topic_word_counts[new_topic][word] += 1
            topic_counts[new_topic] += 1
            document_lengths[d] += 1

100%|██████████| 1000/1000 [00:00<00:00, 1678.48it/s]


In [40]:
for k, word_counts in enumerate(topic_word_counts):
    for word, count in word_counts.most_common():
        if count > 0:
            print(k, word, count)

0 Java 3
0 Big Data 3
0 Hadoop 2
0 HBase 1
0 C++ 1
0 Spark 1
0 Storm 1
0 programming languages 1
0 MapReduce 1
0 Cassandra 1
0 deep learning 1
1 HBase 2
1 neural networks 2
1 Postgres 2
1 MongoDB 2
1 machine learning 2
1 Cassandra 1
1 numpy 1
1 decision trees 1
1 deep learning 1
1 databases 1
1 MySQL 1
1 NoSQL 1
1 artificial intelligence 1
1 scipy 1
2 regression 3
2 Python 2
2 R 2
2 libsvm 2
2 scikit-learn 2
2 mathematics 1
2 support vector machines 1
2 Haskell 1
2 Mahout 1
3 statistics 3
3 probability 3
3 Python 2
3 R 2
3 pandas 2
3 statsmodels 2
3 C++ 1
3 artificial intelligence 1
3 theory 1


In [41]:
topic_names = ["Big Data and programming languages",
               "Python and statistics",
               "databases",
               "machine learning"]

In [42]:
for document, topic_counts in zip(documents, document_topic_counts):
    print(document)
    for topic, count in topic_counts.most_common():
        if count > 0:
            print(topic_names[topic], count)
    print()

['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra']
Big Data and programming languages 7

['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres']
Python and statistics 5

['Python', 'scikit-learn', 'scipy', 'numpy', 'statsmodels', 'pandas']
Python and statistics 2
databases 2
machine learning 2

['R', 'Python', 'statistics', 'regression', 'probability']
machine learning 3
databases 2

['machine learning', 'regression', 'decision trees', 'libsvm']
databases 2
Python and statistics 2

['Python', 'R', 'Java', 'C++', 'Haskell', 'programming languages']
databases 3
Big Data and programming languages 3

['statistics', 'probability', 'mathematics', 'theory']
machine learning 3
databases 1

['machine learning', 'scikit-learn', 'Mahout', 'neural networks']
databases 2
Python and statistics 2

['neural networks', 'deep learning', 'Big Data', 'artificial intelligence']
Python and statistics 3
Big Data and programming languages 1

['Hadoop', 'Java', 'MapReduce', 'Big Data']
Big D

### Word vectors

In [43]:
from scratch.linear_algebra import dot, Vector
import math

def cosine_similarity(v1: Vector, v2: Vector) -> float:
    return dot(v1, v2) / math.sqrt(dot(v1, v1) * dot(v2, v2))

assert cosine_similarity([1., 1, 1], [2., 2, 2]) == 1, "same direction"
assert cosine_similarity([-1., -1], [2., 2]) == -1,    "opposite direction"
assert cosine_similarity([1., 0], [0., 1]) == 0,       "orthogonal"

In [44]:
v1 = [-1, -1]
v2 = [2, 2]
print(dot(v1, v2))

vv1 = dot(v1, v1)
vv2 = dot(v2, v2)
print(vv1)
print(vv2)
math.sqrt(vv1 * vv2)

-4
2
8


4.0

In [45]:
import random

colors = ["red", "green", "blue", "yellow", "black", ""]
nouns = ["bed", "car", "boat", "cat"]
verbs = ["is", "was", "seems"]
adverbs = ["very", "quite", "extremely", ""]
adjectives = ["slow", "fast", "soft", "hard"]

def make_sentence() -> str:
    return " ".join([
        "The",
        random.choice(colors),
        random.choice(nouns),
        random.choice(verbs),
        random.choice(adverbs),
        random.choice(adjectives),
        "."
    ])

NUM_SENTENCES = 50

random.seed(0)
sentences = [make_sentence() for _ in range(NUM_SENTENCES)]

In [46]:
from scratch.deep_learning import Tensor

class Vocabulary:
    def __init__(self, words: List[str] = None) -> None:
        self.w2i: Dict[str, int] = {}  # mapping word -> word_id
        self.i2w: Dict[int, str] = {}  # mapping word_id -> word

        for word in (words or []):     # If words were provided,
            self.add(word)             # add them.

    @property
    def size(self) -> int:
        """how many words are in the vocabulary"""
        return len(self.w2i)

    def add(self, word: str) -> None:
        if word not in self.w2i:        # If the word is new to us:
            word_id = len(self.w2i)     # Find the next id.
            self.w2i[word] = word_id    # Add to the word -> word_id map.
            self.i2w[word_id] = word    # Add to the word_id -> word map.

    def get_id(self, word: str) -> int:
        """return the id of the word (or None)"""
        return self.w2i.get(word)

    def get_word(self, word_id: int) -> str:
        """return the word with the given id (or None)"""
        return self.i2w.get(word_id)

    def one_hot_encode(self, word: str) -> Tensor:
        word_id = self.get_id(word)
        assert word_id is not None, f"unknown word {word}"

        return [1.0 if i == word_id else 0.0 for i in range(self.size)]

<Figure size 432x288 with 0 Axes>

In [47]:
vocab = Vocabulary(["a", "b", "c"])
assert vocab.size == 3,              "there are 3 words in the vocab"
assert vocab.get_id("b") == 1,       "b should have word_id 1"
assert vocab.one_hot_encode("b") == [0, 1, 0]
assert vocab.get_id("z") is None,    "z is not in the vocab"
assert vocab.get_word(2) == "c",     "word_id 2 should be c"
vocab.add("z")
assert vocab.size == 4,              "now there are 4 words in the vocab"
assert vocab.get_id("z") == 3,       "now z should have id 3"
assert vocab.one_hot_encode("z") == [0, 0, 0, 1]

In [48]:
import json

def save_vocab(vocab: Vocabulary, filename: str) -> None:
    with open(filename, 'w') as f:
        json.dump(vocab.w2i, f)       # Only need to save w2i

def load_vocab(filename: str) -> Vocabulary:
    vocab = Vocabulary()
    with open(filename) as f:
        # Load w2i and generate i2w from it
        vocab.w2i = json.load(f)
        vocab.i2w = {id: word for word, id in vocab.w2i.items()}
    return vocab

In [53]:
from typing import Iterable
from scratch.deep_learning import Layer, Tensor, random_tensor, zeros_like

class Embedding(Layer):
    def __init__(self, num_embeddings: int, embedding_dim: int) -> None:
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim

        # One vector of size embedding_dim for each desired embedding
        self.embeddings = random_tensor(num_embeddings, embedding_dim)
        self.grad = zeros_like(self.embeddings)

        # Save last input id
        self.last_input_id = None
        
    def forward(self, input_id: int) -> Tensor:
        """Just select the embedding vector corresponding to the input id"""
        self.input_id = input_id    # remember for use in backpropagation

        return self.embeddings[input_id]
    
    def backward(self, gradient: Tensor) -> None:
        # Zero out the gradient corresponding to the last input.
        # This is way cheaper than creating a new all-zero tensor each time.
        if self.last_input_id is not None:
            zero_row = [0 for _ in range(self.embedding_dim)]
            self.grad[self.last_input_id] = zero_row

        self.last_input_id = self.input_id
        self.grad[self.input_id] = gradient
        
    def params(self) -> Iterable[Tensor]:
        return [self.embeddings]

    def grads(self) -> Iterable[Tensor]:
        return [self.grad]

In [54]:
class TextEmbedding(Embedding):
    def __init__(self, vocab: Vocabulary, embedding_dim: int) -> None:
        # Call the superclass constructor
        super().__init__(vocab.size, embedding_dim)

        # And hang onto the vocab
        self.vocab = vocab
        
    def __getitem__(self, word: str) -> Tensor:
        word_id = self.vocab.get_id(word)
        if word_id is not None:
            return self.embeddings[word_id]
        else:
            return None
        
    def closest(self, word: str, n: int = 5) -> List[Tuple[float, str]]:
        """Returns the n closest words based on cosine similarity"""
        vector = self[word]

        # Compute pairs (similarity, other_word), and sort most similar first
        scores = [(cosine_similarity(vector, self.embeddings[i]), other_word)
                  for other_word, i in self.vocab.w2i.items()]
        scores.sort(reverse=True)

        return scores[:n]

In [58]:
import re

# This is not a great regex, but it works on our data.
tokenized_sentences = [re.findall("[a-z]+|[.]", sentence.lower())
                       for sentence in sentences]
tokenized_sentences[:2]

[['the', 'yellow', 'cat', 'is', 'extremely', 'hard', '.'],
 ['the', 'yellow', 'boat', 'was', 'extremely', 'fast', '.']]

In [61]:
# Create a vocabulary (that is, a mapping word -> word_id) based on our text.
vocab = Vocabulary(word
                   for sentence_words in tokenized_sentences
                   for word in sentence_words)

In [62]:
from scratch.deep_learning import Tensor, one_hot_encode

inputs: List[int] = []
targets: List[Tensor] = []

for sentence in tokenized_sentences:
    for i, word in enumerate(sentence):          # For each word
        for j in [i - 2, i - 1, i + 1, i + 2]:   # take the nearby locations
            if 0 <= j < len(sentence):           # that aren't out of bounds
                nearby_word = sentence[j]        # and get those words.

                # Add an input that's the original word_id
                inputs.append(vocab.get_id(word))

                # Add a target that's the one-hot-encoded nearby word
                targets.append(vocab.one_hot_encode(nearby_word))

In [63]:
from scratch.deep_learning import Sequential, Linear

random.seed(0)
EMBEDDING_DIM = 5  # seems like a good size

# Define the embedding layer separately, so we can reference it.
embedding = TextEmbedding(vocab=vocab, embedding_dim=EMBEDDING_DIM)

model = Sequential([
    # Given a word (as a vector of word_ids), look up its embedding.
    embedding,
    # And use a linear layer to compute scores for "nearby words."
    Linear(input_dim=EMBEDDING_DIM, output_dim=vocab.size)
])

In [64]:
from scratch.deep_learning import SoftmaxCrossEntropy, Momentum, GradientDescent

loss = SoftmaxCrossEntropy()
optimizer = GradientDescent(learning_rate=0.01)

for epoch in range(100):
    epoch_loss = 0.0
    for input, target in zip(inputs, targets):
        predicted = model.forward(input)
        epoch_loss += loss.loss(predicted, target)
        gradient = loss.gradient(predicted, target)
        model.backward(gradient)
        optimizer.step(model)
    print(epoch, epoch_loss)            # Print the loss
    print(embedding.closest("black"))   # and also a few nearest words
    print(embedding.closest("slow"))    # so we can see what's being
    print(embedding.closest("car"))     # learned.

0 2970.156429387063
[(1.0, 'black'), (0.7927247753692507, 'blue'), (0.5911733314896048, 'cat'), (0.5716654981660781, 'the'), (0.3026892975059721, 'car')]
[(1.0, 'slow'), (0.8936272640937163, 'green'), (0.5272141409772461, '.'), (0.356013304388346, 'yellow'), (0.29982973123116025, 'blue')]
[(1.0, 'car'), (0.7030889033452818, 'cat'), (0.519105254858335, 'fast'), (0.4236692673437953, 'very'), (0.3848186865089063, 'quite')]
1 2865.855510833289
[(1.0, 'black'), (0.8076054877794636, 'blue'), (0.6112459000264461, 'cat'), (0.5676562111596322, 'the'), (0.3273307461476133, 'car')]
[(1.0, 'slow'), (0.8795208130766505, 'green'), (0.5675261127213351, '.'), (0.3624106663078739, 'yellow'), (0.3593451263736934, 'boat')]
[(1.0, 'car'), (0.7121322616680856, 'cat'), (0.49717481866583696, 'fast'), (0.4099921723101964, 'very'), (0.3655326707405854, 'quite')]
2 2815.035526656466
[(1.0, 'black'), (0.8240340241906855, 'blue'), (0.6310891601317347, 'cat'), (0.5448895203899405, 'the'), (0.3541498544488745, 'car

20 2504.5902126634855
[(1.0, 'black'), (0.957576053669443, 'blue'), (0.6754448002775352, 'yellow'), (0.6741016113281826, 'red'), (0.6370593374009472, 'cat')]
[(1.0, 'slow'), (0.8669876605786818, 'hard'), (0.7898461908470056, 'soft'), (0.7431142187795304, 'fast'), (0.6054838693034814, 'quite')]
[(1.0, 'car'), (0.8309314677594278, 'cat'), (0.41577966556459073, 'black'), (0.2950449738583089, 'bed'), (0.25992691986485333, 'blue')]
21 2498.7143878987954
[(1.0, 'black'), (0.9603082452264213, 'blue'), (0.6842899903173367, 'yellow'), (0.6819036185885786, 'red'), (0.6116210913792692, 'cat')]
[(1.0, 'slow'), (0.8684111458357192, 'hard'), (0.792082223341027, 'soft'), (0.7524810526902487, 'fast'), (0.6096628372717496, 'quite')]
[(1.0, 'car'), (0.8415691158802513, 'cat'), (0.40780015696694516, 'black'), (0.3138234920502818, 'bed'), (0.2526952060710574, 'blue')]
22 2493.3795337812226
[(1.0, 'black'), (0.9627876259396498, 'blue'), (0.6923235655936569, 'yellow'), (0.6887760594265789, 'red'), (0.607418

39 2447.0824036457702
[(1.0, 'black'), (0.9825567006914961, 'blue'), (0.7685041388865602, 'yellow'), (0.7320503872193427, 'red'), (0.7165520635306126, 'green')]
[(1.0, 'slow'), (0.8805812870264129, 'hard'), (0.8179622958405542, 'fast'), (0.8128659768571114, 'soft'), (0.621173410211933, 'quite')]
[(1.0, 'car'), (0.9224494425335463, 'cat'), (0.49567092149459063, 'bed'), (0.33553771910956887, 'black'), (0.3059141909676227, 'boat')]
40 2445.435059175928
[(1.0, 'black'), (0.9830297405543987, 'blue'), (0.771540671956823, 'yellow'), (0.7326795304623553, 'red'), (0.7208372338967365, 'green')]
[(1.0, 'slow'), (0.880458120067408, 'hard'), (0.8195147526151283, 'fast'), (0.8130554630744775, 'soft'), (0.6193644198955738, 'quite')]
[(1.0, 'car'), (0.9239812945585655, 'cat'), (0.5021638639789027, 'bed'), (0.3327482008616001, 'black'), (0.32542179659037257, 'boat')]
41 2443.829764277178
[(1.0, 'black'), (0.9834644324611019, 'blue'), (0.7745235202523253, 'yellow'), (0.7332236523136183, 'red'), (0.72498

59 2421.4074772212234
[(1.0, 'black'), (0.9870021395567539, 'blue'), (0.8207439713993813, 'yellow'), (0.78393730742721, 'green'), (0.7347330133673399, 'red')]
[(1.0, 'slow'), (0.8752828866686732, 'hard'), (0.848157353519796, 'fast'), (0.8189279414780526, 'soft'), (0.5487288039931084, 'quite')]
[(1.0, 'car'), (0.9157500668927478, 'cat'), (0.6192307192501065, 'bed'), (0.5665359263949067, 'boat'), (0.2775430564257161, 'black')]
60 2420.50423862855
[(1.0, 'black'), (0.9870290148847191, 'blue'), (0.8228741772663025, 'yellow'), (0.7865738299248013, 'green'), (0.7345677788547076, 'red')]
[(1.0, 'slow'), (0.8750945811724076, 'hard'), (0.8496294491509623, 'fast'), (0.8194974712260712, 'soft'), (0.5439055883980406, 'quite')]
[(1.0, 'car'), (0.9139163013482309, 'cat'), (0.6253693669470449, 'bed'), (0.5730029937205466, 'boat'), (0.27508395082824433, 'black')]
61 2419.6331284472003
[(1.0, 'black'), (0.9870424596759124, 'blue'), (0.8249545122095633, 'yellow'), (0.7891568805009261, 'green'), (0.73438

80 2408.2135766837687
[(1.0, 'black'), (0.9854148512712754, 'blue'), (0.8552991079179565, 'yellow'), (0.8293099393854022, 'green'), (0.7292971266782381, 'red')]
[(1.0, 'slow'), (0.8768065331415691, 'hard'), (0.8751113979543328, 'fast'), (0.8349419301515192, 'soft'), (0.45485178519148356, 'quite')]
[(1.0, 'car'), (0.8785236145922348, 'cat'), (0.7356350107537116, 'bed'), (0.656620258923684, 'boat'), (0.24149321116016956, 'black')]
81 2407.8320561830847
[(1.0, 'black'), (0.9852615735169723, 'blue'), (0.8564474543662007, 'yellow'), (0.8310084376448683, 'green'), (0.7289484754345525, 'red')]
[(1.0, 'slow'), (0.8771346630632108, 'hard'), (0.8761277442755065, 'fast'), (0.8357997951621013, 'soft'), (0.4511801643389558, 'quite')]
[(1.0, 'car'), (0.8770581043441031, 'cat'), (0.7400342517800104, 'bed'), (0.6594825896382437, 'boat'), (0.2404320975323036, 'black')]
82 2407.4673830822285
[(1.0, 'black'), (0.9851043112165103, 'blue'), (0.8575554595630774, 'yellow'), (0.8326706354710252, 'green'), (0.

99 2403.2259775721222
[(1.0, 'black'), (0.9821293900663808, 'blue'), (0.870937253624594, 'yellow'), (0.8560680073921026, 'green'), (0.720997727387519, 'red')]
[(1.0, 'slow'), (0.8900872652719088, 'fast'), (0.8846955683864671, 'hard'), (0.8505709819185356, 'soft'), (0.3979688002551062, 'quite')]
[(1.0, 'car'), (0.8545578601895444, 'cat'), (0.7977433731597632, 'bed'), (0.6982593591613971, 'boat'), (0.228384707903377, 'black')]


In [65]:
pairs = [(cosine_similarity(embedding[w1], embedding[w2]), w1, w2)
         for w1 in vocab.w2i
         for w2 in vocab.w2i
         if w1 < w2]
pairs.sort(reverse=True)
print(pairs[:5])

[(0.9948304450324577, 'bed', 'cat'), (0.9920899253848237, 'seems', 'was'), (0.9883988811665564, 'bed', 'boat'), (0.9821293900663808, 'black', 'blue'), (0.9709525703302306, 'green', 'red')]


In [72]:
# ???
# from scratch.working_with_data import pca, transform
# import matplotlib.pyplot as plt

# # Extract the first two principal components and transform the word vectors
# components = pca(embedding.embeddings, 2)
# transformed = transform(embedding.embeddings, components)

# # Scatter the points (and make them white so they're "invisible")
# fig, ax = plt.subplots()
# ax.scatter(*zip(*transformed), marker='.', color='w')

# # Add annotations for each word at its transformed location
# for word, idx in vocab.w2i.items():
#     ax.annotate(word, transformed[idx])

# # And hide the axes
# ax.get_xaxis().set_visible(False)
# ax.get_yaxis().set_visible(False)

# plt.show()

### Recurrent Neural Networks

In [76]:
from scratch.deep_learning import tensor_apply, tanh

class SimpleRnn(Layer):
    """Just about the simplest possible recurrent layer."""
    def __init__(self, input_dim: int, hidden_dim: int) -> None:
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.w = random_tensor(hidden_dim, input_dim, init='xavier')
        self.u = random_tensor(hidden_dim, hidden_dim, init='xavier')
        self.b = random_tensor(hidden_dim)

        self.reset_hidden_state()

    def reset_hidden_state(self) -> None:
        self.hidden = [0 for _ in range(self.hidden_dim)]
        
    def forward(self, input: Tensor) -> Tensor:
        self.input = input              # Save both input and previous
        self.prev_hidden = self.hidden  # hidden state to use in backprop.

        a = [(dot(self.w[h], input) +           # weights @ input
              dot(self.u[h], self.hidden) +     # weights @ hidden
              self.b[h])                        # bias
             for h in range(self.hidden_dim)]

        self.hidden = tensor_apply(tanh, a)  # Apply tanh activation
        return self.hidden                   # and return the result.
    
    def backward(self, gradient: Tensor):
        # Backpropagate through the tanh
        a_grad = [gradient[h] * (1 - self.hidden[h] ** 2)
                  for h in range(self.hidden_dim)]

        # b has the same gradient as a
        self.b_grad = a_grad

        # Each w[h][i] is multiplied by input[i] and added to a[h],
        # so each w_grad[h][i] = a_grad[h] * input[i]
        self.w_grad = [[a_grad[h] * self.input[i]
                        for i in range(self.input_dim)]
                       for h in range(self.hidden_dim)]

        # Each u[h][h2] is multiplied by hidden[h2] and added to a[h],
        # so each u_grad[h][h2] = a_grad[h] * prev_hidden[h2]
        self.u_grad = [[a_grad[h] * self.prev_hidden[h2]
                        for h2 in range(self.hidden_dim)]
                       for h in range(self.hidden_dim)]

        # Each input[i] is multiplied by every w[h][i] and added to a[h],
        # so each input_grad[i] = sum(a_grad[h] * w[h][i] for h in ...)
        return [sum(a_grad[h] * self.w[h][i] for h in range(self.hidden_dim))
                for i in range(self.input_dim)]
    
    def params(self) -> Iterable[Tensor]:
        return [self.w, self.u, self.b]

    def grads(self) -> Iterable[Tensor]:
        return [self.w_grad, self.u_grad, self.b_grad]